# Εργαστήριο 13
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Εντολές ανάγνωσης/εγγραφής (load/store) στη μνήμη δεδομένων
Στο εργαστήριο αυτό θα ολοκληρώσετε τις προσθήκες στα υπάρχοντα modules για να υλοποιήσετε τις εντολές **ανάγνωσης (load)** και **εγγραφής (store)** μιας λέξης στη μνήμη δεδομένων (data memory).

Η υλοποίηση γίνεται σε 2 μέρη:

* Στο προηγούμενο εργαστήριο τροποποιήσατε τα modules `FetchUnit` και `DataPath` για να προσθέσετε τη **μνήμη δεδομένων** στο σύστημα.

* Στο παρόν εργαστήριο θα προσθέσετε λειτουργικότητα στα modules `DecodeUnit` και `Cpu` για την **εκτέλεση των εντολών load και store**.

Στις επόμενες ενότητες περιγράφεται ξανά η κωδικοποίηση και η αναμενόμενη λειτουργία των εντολών load και store.

### Εντολή load (ανάγνωση από τη μνήμη δεδομένων)

`load Dst,Rsrc2,Offset`

Η εντολή αυτή θα έχει την εξής λειτουργικότητα:

`Dst <- memory[address = Rsrc2 + Offset]`

όπου είναι:

* `Dst`: καταχωρητής προορισμού (3 bits)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (3 bits)
* `Offset`: σταθερά (ακέραιος με πρόσημο, 7 bits), προστίθεται στο περιεχόμενο του Rsrc2 για το σχηματισμό της διεύθυνσης μνήμης για ανάγνωση

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-13` | `bits 12-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- |
| `011` | `Offset[6:0]` | `Rsrc2` | `Dst` |


### Εντολή store (εγγραφή στη μνήμη δεδομένων)

`store Rsrc1,Rsrc2,Offset`

Η εντολή αυτή θα έχει την εξής λειτουργικότητα:

`memory[address = Rsrc2 + Offset] <- Rsrc1`

όπου είναι:

* `Rsrc1`: καταχωρητής εισόδου δεδομένων 1 (3 bits)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (3 bits)
* `Offset`: σταθερά (ακέραιος με πρόσημο, 7 bits), προστίθεται στο περιεχόμενο του Rsrc2 για το σχηματισμό της διεύθυνσης μνήμης για ανάγνωση

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-13` | `bits 12-9` | `bits 8-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- | --- |
| `111` | `Offset[6:3]` | `Rsrc2` | `Rsrc1` | `Offset[2:0]` |


## Κώδικας από προηγούμενα εργαστήρια
Στα παρακάτω κελιά **αντιγράψτε χωρίς αλλαγές** τα αντίστοιχα modules που έχετε ήδη από προηγούμενα εργαστήρια.

In [ ]:
class InstructionMemory(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class FetchUnit(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class RegisterFile(register_width: Int, register_number: Int) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class Alu(n: Int) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class DataMemory(word_number: Int,word_width: Int) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class DataPath(word_width: Int, register_number: Int, memory_size: Int) extends Module {

  // ..συμπληρώστε..  
    
}

In [ ]:
class BranchLogic extends Module {

  // ..συμπληρώστε..  
    
}

## Άσκηση 1: προσθήκες στο module DecodeUnit
Με την προσθήκη της εντολής `load` εμφανίζεται η εξής ιδιαιτερότητα: ενώ όλα τα προηγούμενα είδη εντολών (αλλά και η `store`) ολοκληρώνονται σε έναν κύκλο ρολογιού η καθεμία, η εντολή `load` χρειάζεται 2 κύκλους ρολογιού για να ολοκληρωθεί.
* Στον 1ο κύκλο ολοκληρώνονται οι φάσεις Instruction Fetch (IF, λήψη εντολής), Instruction Decode (ID, προετοιμασία σημάτων και επιλογή πηγών) και Execute (EX, υπολογίζει τη διεύθυνση ανάγνωσης).
* Στον 2ο κύκλο ολοκηρώνονται οι φάσεις Data Memory (DM, ανάγνωση από μνήμη) και (στο τέλος του κύκλου) η Write Back (WB, εγγραφή αποτελέσματος σε καταχωρητή).

Συνεπώς απαιτείται η τήρηση **κατάστασης (state)** για τη φάση που βρίσκεται η εντολή load.

Ξεκινήστε από την προηγούμενη υλοποίηση του `DecodeUnit` που έχετε, προσθέτοντας τα παρακάτω:

1. Αρχικά προσθέστε τις 3 *εξόδους* ελέγχου που παράγονται στο `DecodeUnit`:
  * `freeze_pc` (εύρος 1 bit), όταν είναι 1 δεν ενημερώνεται ο pc με νέα τιμή στο `FetchUnit`
  * `result_sel` (εύρος 1 bit), επιλογή τελικού αποτελέσματος στο `DataPath`(0 = από alu, 1 = από μνήμη δεδομένων)
  * `write_en` (εύρος 1 bit), αν είναι 1, η μνήμη δεδομένων στο `DataPath` εκτελεί εγγραφή.
  
  
2. Αφού τελειώσει ο ορισμός του bundle `io` με τις εισόδους/εξόδους, προσθέστε έναν καταχωρητή του 1 bit, ο οποίος θα κρατά την κατάσταση της φάσης που βρίσκεται η `load` (0 = 1ος κύκλος, 1 = 2ος κύκλος, αρχική κατάσταση 0):

  ~~~scala
   val state = RegInit(0.U(1.W))
  ~~~
  
3. Από την κωδικοποίηση των εντολών που δόθηκε στην αρχή της εκφώνησης, φαίνεται ότι η εντολή `store` χρησιμοποιεί το `offset1`, όπως αυτό έχει ήδη οριστεί μέσα στο `DecodeUnit`. Το offset όμως που χρησιμοποιεί η `load` είναι διαφορετικό, προσθέστε το μετά τον ορισμό του `offset1`:

  ~~~scala
  val offset2 = io.instruction(12,6).asSInt.pad(16).asUInt
  ~~~
  
4. Προσθέστε την υλοποίηση της εντολής store:

  ~~~scala
  is("b111".U) {  // store
      // defaults used: read_sel_a, read_sel_b, write_sel=0, sub=0
      io.alu_a_sel :=     // ..συμπληρώστε..
      io.im :=            // ..συμπληρώστε..
      io.sel :=           // ..συμπληρώστε..
      io.write_en :=      // ..συμπληρώστε.. 
  } 
  ~~~
  
  Η υλοποίηση της `store` θα πρέπει να οδηγεί κατάλληλα τις εξόδους `alu_a_sel`, `im`, `sel` και `write_en` έτσι ώστε το `DataPath`:
  * Να προσθέτει (+) το `offset1` με τα δεδομένα του καταχωρητή που επιλέγει το `read_sel_b`.
  * Να επιτρέπει την εγγραφή στη μνήμη δεδομένων του περιεχομένου του καταχωρητή που επιλέγει το `read_sel_a`.
  
  
5. Προσθέστε την υλοποίηση της εντολής load σε δύο φάσεις:

  ~~~scala
  is("b011".U) {  // load (2-cycle instr)
    when (state === 0.U) {  // cycle 1, compute address
      state := 1.U
      io.freeze_pc := 1.U
      // defaults used: read_sel_b, write_sel=0, sub=0
      io.alu_a_sel :=     // ..συμπληρώστε..
      io.im :=            // ..συμπληρώστε..
      io.sel :=           // ..συμπληρώστε..
          
    }.otherwise { // cycle 2, read mem and (prepare to) store to reg
      state := 0.U
      // defaults used: - 
      io.write_sel :=     // ..συμπληρώστε..
      io.result_sel :=    // ..συμπληρώστε.. 
    }  
  }
  ~~~
  
  Στον 1ο κύκλο η υλοποίηση της load:
  * Περνάει στη 2η φάση (state = 1)
  * Οδηγεί το `freeze_pc` στην τιμή 1, έτσι ώστε να μην προχωρήσουμε στην επόμενη εντολή.
  * Οδηγήστε κατάλληλα τα σήματα `alu_a_sel`, `im` και `sel` έτσι ώστε το `DataPath` να προσθέτει (+) το `offset2` με τα δεδομένα του καταχωρητή που επιλέγει το `read_sel_b`.
  
  Στον 2ο κύκλο η υλοποίηση της load:
  * Περνάει στη 1η φάση (state = 0)
  * Οδηγήστε κατάλληλα τα σήματα `result_sel` και `write_sel` έτσι ώστε το `DataPath`:
    * Να επιλέγει αποτέλεσμα από την έξοδο της μνήμης δεδομένων (όχι την alu).
    * Να αποθηκεύει το αποτέλεσμα στον καταχωρητή που επιλέγουν τα bits 2..0 του `instruction`.
  

In [ ]:
class DecodeUnit extends Module {

  // ..συμπληρώστε..  
    
}

Δοκιμάστε την ορθή λειτουργία του module `DecodeUnit`:

In [ ]:
test(new DecodeUnit) { c =>
  // test logic-arithmetic instr
  c.io.instruction.poke("b0000011001010011".U)  // r3 = r1 + r2
  c.io.write_en.expect(0.U)
  c.io.result_sel.expect(0.U)
  c.io.freeze_pc.expect(0.U)  
  // test load immediate instr  
  c.io.instruction.poke("b0010000100001001".U)  // r1 = 33
  c.io.write_en.expect(0.U)
  c.io.result_sel.expect(0.U)
  c.io.freeze_pc.expect(0.U)  
  // test branch_if_equal instr  
  c.io.instruction.poke("b0100000001010100".U)  // if r1 == r2 then pc += 4
  c.io.write_en.expect(0.U)
  c.io.result_sel.expect(0.U)
  c.io.freeze_pc.expect(0.U)  
  // test store instr  
  c.io.instruction.poke("b1110001001011111".U)  // mem[r3+15] = r1
  c.io.read_sel_a.expect(1.U)
  c.io.read_sel_b.expect(3.U)
  c.io.alu_a_sel.expect(0.U)
  c.io.im.expect(15.U)
  c.io.sel.expect("b11".U)
  c.io.sub.expect(0.U)
  c.io.write_sel.expect(0.U)  
  c.io.write_en.expect(1.U)
  c.io.result_sel.expect(0.U)
  c.io.freeze_pc.expect(0.U)
  // test load instr
  c.io.instruction.poke("b0111111111111100".U)  // r4 = mem[r7-1]
  c.io.read_sel_b.expect(7.U)
  c.io.alu_a_sel.expect(0.U)
  c.io.im.expect("b1111111111111111".U)
  c.io.sel.expect("b11".U)
  c.io.sub.expect(0.U)
  c.io.write_sel.expect(0.U)    
  c.io.write_en.expect(0.U)
  c.io.result_sel.expect(0.U)
  c.io.freeze_pc.expect(1.U)
  c.clock.step()  
  c.io.write_sel.expect(4.U)    
  c.io.write_en.expect(0.U)
  c.io.result_sel.expect(1.U)
  c.io.freeze_pc.expect(0.U)
        
}
println("SUCCESS!!")

## Άσκηση 2: Η νέα μορφή του module Cpu
Προσθέστε στο module `Cpu` που έχετε από προηγούμενα εργαστήρια τις συνδέσεις των νέων σημάτων:

1. Το `freeze_pc` (έξοδος από το `DecodeUnit`, είσοδος στο `FetchUnit`).
2. Το `write_en` (έξοδος από το `DecodeUnit`, είσοδος στο `DataPath`).
3. Το `result_sel` (έξοδος από το `DecodeUnit`, είσοδος στο `DataPath`).

In [ ]:
class Cpu(instructions: Seq[UInt]) extends Module {

  // ..συμπληρώστε..  
    
}

Δοκιμάστε το νέο module `Cpu` εκτελώντας το παρακάτω:

In [ ]:

val instructions = List("b0010000001010001".U,  // r1 = 10       (limit = 10)
                        "b0010000000000010".U,  // r2 = 0        (sum = 0)
                        "b0010000000001011".U,  // r3 = 1        (counter = 1)
                        "b0010000000001100".U,  // r4 = 1        (constant 1)
                        "b0000011010011010".U,  // r2 = r2 + r3  (sum = sum + counter)
                        "b1110000010011000".U,  // mem[r3+0] = r2
                        "b0000011011100011".U,  // r3 = r3 + r4  (counter++)
                        "b0000111001100001".U,  // r1 = r1 - r4  (limit--)
                        "b0100000001000001".U,  // beq r1,r0,+1  (if limit==0 then pc <- (pc+1)+1)
                        "b0101111000000010".U,  // beq r0,r0,-6  (pc <- (pc+1)-6)
                        "b0000001010000000".U,  // r0 = r2 or r0 (to show r2 contents, total sum 1+2+3..+10)
                        // second part of program
                        "b0010000000001011".U,  // r3 = 1        (counter = 1)
                        "b0010000001011001".U,  // r1 = 11       (limit = 11)
                        "b0110000000011000".U,  // r0 = mem[r3+0](to show mem contents)
                        "b0000011011100011".U,  // r3 = r3 + r4  (counter++)
                        "b0100000001011001".U,  // beq r1,r3,+1  (if counter==limit then pc <- (pc+1)+1)
                        "b0101111000000100".U,  // beq r0,r0,-4  (pc <- (pc+1)-4)
                        "b0011111111111000".U,  // r0 = 0xFFFF   (show 0xFFFF marker)
                       )

test(new Cpu(instructions)) { c =>
  // first part of program, compute and store to memory partial sums from 1 to 10  
  for (i <- 0 until 63) {
    //println(c.io.results.peek())  
    c.clock.step()
  }
  c.io.results.expect(55.U) // sum(1..10)
  
  // second part of program, retrieve from memory partial sums from 1 to 10
  c.clock.step()
  c.clock.step()  
  for (i <- 0 until 10) {
    c.clock.step()  // 2 cycles for load instruction
    c.clock.step()  
    println(c.io.results.peek())  
    c.clock.step()
    c.clock.step()
    c.clock.step()  
  }
  c.io.results.expect(65535.U)   // 0xFFFF, reached end of program
}
println("SUCCESS!!")

## Ανεβάστε τη λύση σας στο opencourses
Ανεβάστε το τελικό σας notebook στο opencourses (**Εργασία 4**) έως και τη **Δευτέρα 30/1**.

*(Ίσως χρειαστεί να κάνετε zip το αρχείο ipynb για να γίνει αποδεκτό από το σύστημα)*
